In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "<openai_api_key>"
os.environ["TRAVERSAAL_ARES_API_KEY"] = "<traversaal_ares_api_key>"

In [2]:
from agentpro import AgentPro
from agentpro.tools import AresInternetTool, CodeEngine, YouTubeSearchTool, SlideGenerationTool
import os

def main():
    tools = [
        AresInternetTool(),
        CodeEngine(), 
        YouTubeSearchTool(), 
        SlideGenerationTool()
    ]
    
    # Create agent with tools
    agent = AgentPro(tools=tools)
    
    print("AgentPro is initialized and ready. Enter 'quit' to exit.")
    print("Available tools:")
    for tool in tools:
        print(f"- {tool.name}: {tool.description}")
    
    while True:
        user_input = input("\nEnter your query: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            break
            
        try:
            response = agent(user_input)
            print(f"\nAgent Response:\n{response}")
        except Exception as e:
            print(f"Error: {e}")

if __name__ == "__main__":
    main()

Tools being registered:
- Original name: 'ares_internet_search_tool', Normalized: 'ares_internet_search_tool'
- Original name: 'code_generation_and_execution_tool', Normalized: 'code_generation_and_execution_tool'
- Original name: 'youtube_search_tool', Normalized: 'youtube_search_tool'
- Original name: 'slide_generation_tool', Normalized: 'slide_generation_tool'
Registered tools:
- Key: 'ares_internet_search_tool', Tool name: 'ares_internet_search_tool'
- Key: 'code_generation_and_execution_tool', Tool name: 'code_generation_and_execution_tool'
- Key: 'youtube_search_tool', Tool name: 'youtube_search_tool'
- Key: 'slide_generation_tool', Tool name: 'slide_generation_tool'
AgentPro is initialized and ready. Enter 'quit' to exit.
Available tools:
- ares_internet_search_tool: tool to search real-time relevant content from the internet. use this tool for any factual information or recent events.
- code_generation_and_execution_tool: a coding tool that can take a prompt and generate execut


Enter your query:  q


### Custom Tool

In [3]:
from agentpro.tools.base import Tool
from huggingface_hub import list_models

class MostModelTool(Tool):
    name: str = "model_download_tool"
    description: str = (
        "Returns the most downloaded model checkpoint on the Hugging Face Hub "
        "for a given task (e.g., 'text-classification', 'translation')."
    )
    arg: str = "The task name for which you want the top model."

    def run(self, prompt: str) -> str:
        task_name = prompt.strip()
        models = list_models(filter=task_name, sort="downloads", direction=-1)
        top_model = next(iter(models))
        return top_model.id

In [5]:
most_model_download_tool = MostModelTool()
ares_tool = AresInternetTool()
code_tool = CodeEngine()

agent2 = AgentPro(tools=[most_model_download_tool, ares_tool, code_tool])

# Define a task (e.g., 'text-generation', 'image-classification', 'text-to-video', 'text-classification')

# Run a query
response = agent2("Can you give me the name of the model that has the most downloads in the 'text-classification' task on the Hugging Face Hub?")
print(response)

Tools being registered:
- Original name: 'model_download_tool', Normalized: 'model_download_tool'
- Original name: 'ares_internet_search_tool', Normalized: 'ares_internet_search_tool'
- Original name: 'code_generation_and_execution_tool', Normalized: 'code_generation_and_execution_tool'
Registered tools:
- Key: 'model_download_tool', Tool name: 'model_download_tool'
- Key: 'ares_internet_search_tool', Tool name: 'ares_internet_search_tool'
- Key: 'code_generation_and_execution_tool', Tool name: 'code_generation_and_execution_tool'

Available tools: ['model_download_tool', 'ares_internet_search_tool', 'code_generation_and_execution_tool']
User prompt: Can you give me the name of the model that has the most downloads in the 'text-classification' task on the Hugging Face Hub?

--- Step 1 ---

Assistant response:
--------------------------------------------------
Thought: The task requires information about the most downloaded model checkpoint on the hugging face hub for the 'text-classifi

In [6]:
import json
from typing import Union
from duckduckgo_search import DDGS
from agentpro.tools.base import Tool

class DuckDuckGoSearchTool(Tool):
    name: str = "DuckDuckGo Search Tool"
    description: str = (
        "A tool that performs a web search using DuckDuckGo. It returns the top 5 search results "
        "with title, description, and URL. Useful for retrieving up-to-date facts, news, or data."
    )
    arg: str = "A search query string. Make sure the Action Input is JSON-parseable if passed as a string."

    def run(self, prompt: Union[str, dict]) -> str:
        print(f"Calling DuckDuckGo Search Tool with prompt TYPE: {type(prompt)}")

        if isinstance(prompt, str):
            try:
                prompt = json.loads(prompt)
                print("Converted prompt from JSON string to dictionary.")
            except json.JSONDecodeError:
                prompt = prompt.strip()
        elif isinstance(prompt, dict):
            prompt = prompt.get("query", "")
        else:
            return "Error: Prompt must be a string or a JSON dictionary with a 'query' key."

        if not prompt:
            return "Error: Query string is empty."

        try:
            with DDGS() as ddgs:
                results = ddgs.text(
                    keywords=prompt,
                    region="wt-wt",
                    safesearch="moderate",
                    max_results=5
                )

                if not results:
                    return "No results found for the query."

                formatted_results = []
                for i, result in enumerate(results, 1):
                    title = result.get('title', 'No title')
                    body = result.get('body', 'No description')
                    href = result.get('href', 'No URL')

                    formatted_result = f"Result {i}:\nTitle: {title}\nDescription: {body}\nURL: {href}\n"
                    formatted_results.append(formatted_result)

                return "\n".join(formatted_results)

        except Exception as e:
            return f"Error performing search: {str(e)}"

# Create an instance of the tool
duckduckgo_search = DuckDuckGoSearchTool()


In [7]:
# Make sure the class is already defined above or imported here
duckduckgo_search = DuckDuckGoSearchTool()

# Run a test query
# query = "Latest breakthroughs in AI 2025"
# response = duckduckgo_search.run(query)

query = "PSL 2025 matches updates with links"
response = duckduckgo_search.run(query)

print("Search Results:\n")
print(response)


Calling DuckDuckGo Search Tool with prompt TYPE: <class 'str'>
Search Results:

Result 1:
Title: PSL 2025: Latest News, Match Schedules, and Live Updates for Pakistan ...
Description: Stay updated with PSL 2025 on IPL.com. Get the latest news, detailed match schedules, live scores, and in-depth analyses of the Pakistan Super League. Follow your favorite teams and players throughout the tournament. ... LATEST UPDATE. IPL 2025: Updated Orange and Purple Cap standings after the RCB vs MI match. by UMMADI JAYA April 8, 2025. 165. 0.
URL: https://www.ipl.com/cricket/series-matches/psl-2025/

Result 2:
Title: Pakistan Super League, 2025 schedule, live scores and results
Description: Pakistan Super League, 2025 Schedule, Match Timings, Venue Details, Upcoming Cricket Matches and Recent Results on Cricbuzz.com ... PSL 2025. Quetta Gladiators vs Peshawar Zalmi. 2nd Match. Multan ...
URL: https://www.cricbuzz.com/cricket-series/9460/pakistan-super-league-2025/matches

Result 3:
Title: PSL 2025: 

In [8]:
from agentpro import AgentPro
from agentpro.tools import CodeEngine

# Initialize your agent with the custom DuckDuckGo search tool
agent = AgentPro(tools=[duckduckgo_search, CodeEngine()])

# Example queries
response = agent("Latest breakthroughs in AI 2025 with links")
print(response)

# response = agent("What are the latest developments in artificial intelligence? with links")
# print(response)

# # Ask about current events
# response = agent("What's happening with renewable energy in Europe?")
# print(response)

Tools being registered:
- Original name: 'duckduckgo_search_tool', Normalized: 'duckduckgo_search_tool'
- Original name: 'code_generation_and_execution_tool', Normalized: 'code_generation_and_execution_tool'
Registered tools:
- Key: 'duckduckgo_search_tool', Tool name: 'duckduckgo_search_tool'
- Key: 'code_generation_and_execution_tool', Tool name: 'code_generation_and_execution_tool'

Available tools: ['duckduckgo_search_tool', 'code_generation_and_execution_tool']
User prompt: Latest breakthroughs in AI 2025 with links

--- Step 1 ---

Assistant response:
--------------------------------------------------
Thought: To find this information, the best approach is to use the duckduckgo_search_tool, since it searches the internet for the most up-to-date information.
Action: duckduckgo_search_tool
Action Input: "AI breakthroughs 2025"
--------------------------------------------------

DEBUG - Tool Call: 'duckduckgo_search_tool'
DEBUG - Tool Input: 'AI breakthroughs 2025'
DEBUG - Using too

In [14]:
from agentpro import AgentPro
from agentpro.tools import CodeEngine

# Initialize your agent with the custom DuckDuckGo search tool
agent = AgentPro(tools=[duckduckgo_search, CodeEngine()])

# Example queries
response = agent("Latest updates of IPL 2025 with links")
print(response)

Tools being registered:
- Original name: 'duckduckgo_search_tool', Normalized: 'duckduckgo_search_tool'
- Original name: 'code_generation_and_execution_tool', Normalized: 'code_generation_and_execution_tool'
Registered tools:
- Key: 'duckduckgo_search_tool', Tool name: 'duckduckgo_search_tool'
- Key: 'code_generation_and_execution_tool', Tool name: 'code_generation_and_execution_tool'

Available tools: ['duckduckgo_search_tool', 'code_generation_and_execution_tool']
User prompt: Latest updates of IPL 2025 with links

--- Step 1 ---

Assistant response:
--------------------------------------------------
Thought: The latest updates on the IPL 2025 are likely to be found through an internet search, as this will provide the most current and relevant information. 
Action: duckduckgo_search_tool
Action Input: "Latest updates of IPL 2025"
--------------------------------------------------

DEBUG - Tool Call: 'duckduckgo_search_tool'
DEBUG - Tool Input: 'Latest updates of IPL 2025'
DEBUG - Usi

In [16]:
ares_tool = AresInternetTool()
code_engine = CodeEngine()

from agentpro import AgentPro
from agentpro.tools import CodeEngine

# Initialize your agent with the custom DuckDuckGo search tool
agent = AgentPro(tools=[ares_tool, code_engine])

# Example queries
response = agent("Latest updates of PSL 2025 with links")
print(response)

Tools being registered:
- Original name: 'ares_internet_search_tool', Normalized: 'ares_internet_search_tool'
- Original name: 'code_generation_and_execution_tool', Normalized: 'code_generation_and_execution_tool'
Registered tools:
- Key: 'ares_internet_search_tool', Tool name: 'ares_internet_search_tool'
- Key: 'code_generation_and_execution_tool', Tool name: 'code_generation_and_execution_tool'

Available tools: ['ares_internet_search_tool', 'code_generation_and_execution_tool']
User prompt: Latest updates of PSL 2025 with links

--- Step 1 ---

Assistant response:
--------------------------------------------------
Thought: The user is asking for the latest updates of PSL (Pakistan Super League) in 2025 with links. As AI was last trained on data up until September 2021, I need real-time information on this topic. Hence, I will use the ares_internet_search_tool.
Action: ares_internet_search_tool
Action Input: "latest updates PSL 2025"
--------------------------------------------------